# OptaPy - OptaPlanner in Python

Optapy is an unofficial and experimental GraalVM Python Package
that allows OptaPlanner Constraint Providers to be written using
only Python code.

WARNING: Optapy is an experimental technology and is not supported in any way or form. Additionally, it is at least 20 times slower than using OptaPlanner in Java. It is not recommended for production use.

## What is OptaPlanner?

OptaPlanner is an AI constraint solver. It optimizes planning and scheduling problems, such as the Vehicle Routing Problem, Employee Rostering, Maintenance Scheduling, Task Assignment, School Timetabling, Cloud Optimization, Conference Scheduling, Job Shop Scheduling, Bin Packing and many more. Every organization faces such challenges: assign a limited set of constrained resources (employees, assets, time and/or money) to provide products or services. OptaPlanner delivers more efficient plans, which reduce costs and improve service quality.

Constraints apply on plain domain objects and can call existing code. There’s no need to input constraints as mathematical equations. Under the hood, OptaPlanner combines sophisticated Artificial Intelligence optimization algorithms (such as Tabu Search, Simulated Annealing, Late Acceptance and other metaheuristics) with very efficient score calculation and other state-of-the-art constraint solving techniques.

## An Example: School Timetabling

### Model the domain objects and constraints

The goal is to assign each lesson to a time slot and a room. The model is divided into four kind of objects

#### Problem Facts

Problem facts are facts about the problem. As such, they do not change during solving (and thus cannot have any planning variables). An example problem fact is shown below:

In [1]:
from optapy import problem_fact, planning_id

@problem_fact
class Room:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    @planning_id
    def getId(self):
        return self.id

    def __str__(self):
        return "Room(id=" + str(self.id) + ", name=" + str(self.name) + ")"

The `@problem_fact` decorator creates a Java class for Room, which allows it to be used in constraints. The `@planning_id` decorator tells OptaPlanner that it can use that method for identifying identifical pairs. It is only required if you use `fromUniquePair` on the class in a constraint.

The code for the Timeslot probelm fact is shown below:

In [2]:
@problem_fact
class Timeslot:
    def __init__(self, id, dayOfWeek, startTime, endTime):
        self.id = id
        self.dayOfWeek = dayOfWeek
        self.startTime = startTime
        self.endTime = endTime

    @planning_id
    def getId(self):
        return self.id

    def __str__(self):
        return "Timeslot(id=" + str(self.id) + \
               ", dayOfWeek=" + str(self.dayOfWeek) + ", startTime=" + str(self.startTime) + \
               ", endTime=" + str(self.endTime) + ")"

#### Planning Entities

During a lesson, represented by the Lesson class, a teacher teaches a subject to a group of students, for example, Math by A.Turing for 9th grade or Chemistry by M.Curie for 10th grade. If a subject is taught multiple times per week by the same teacher to the same student group, there are multiple Lesson instances that are only distinguishable by id. For example, the 9th grade has six math lessons a week.

During solving, OptaPlanner changes the timeslot and room fields of the Lesson class, to assign each lesson to a time slot and a room. Because OptaPlanner changes these fields, Lesson is a planning entity. Here is how we would write it in Python:

In [3]:
from optapy import planning_entity, planning_variable

@planning_entity
class Lesson:
    def __init__(self, id, subject, teacher, studentGroup, timeslot=None, room=None):
        self.id = id
        self.subject = subject
        self.teacher = teacher
        self.studentGroup = studentGroup
        self.timeslot = timeslot
        self.room = room

    @planning_id
    def getId(self):
        return self.id

    @planning_variable(Timeslot, ["timeslotRange"])
    def getTimeslot(self):
        return self.timeslot

    def setTimeslot(self, newTimeslot):
        self.timeslot = newTimeslot

    @planning_variable(Room, ["roomRange"])
    def getRoom(self):
        return self.room

    def setRoom(self, newRoom):
        self.room = newRoom

    def __str__(self):
        return "Lesson(id=" + str(self.id) + \
                ", timeslot=" + str(self.timeslot) + ", room=" + str(self.room) + \
                ", teacher=" + str(self.teacher) + ", subject=" + str(self.subject) + \
                ", studentGroup=" + str(self.studentGroup) + ")"

The `@planning_entity` decorator creates a Java class for Lesson, which allows it to be used in constraints.
The `@planning_variable` specify that a method returns a planning variable. As such, OptaPlanner will call the corresponding setter to change the value of the variable during solving. It must be named `get%Variable()` and has a corresponding setter `set%Variable` (where `%Variable` is the name of the variable). It takes two parameters:

- The first parameter is the type this planning variable takes.
- The second parameter, `value_range_provider_refs`, describes where it gets its values from.
  It a list of the id of its value range providers


#### The Constraints

The constraints tell OptaPlanner how good a solution is. Here how we create the constraints in Python:

In [4]:
from optapy import constraint_provider, get_class
from optapy.types import Joiners, HardSoftScore
from datetime import datetime, date, timedelta

# Constraint Factory takes Java Classes, not Python Classes
LessonClass = get_class(Lesson)
RoomClass = get_class(Room)

# Workaround for python timedelta requiring a date
today = date.today()
def within30Mins(lesson1, lesson2):
    between = datetime.combine(today, lesson1.timeslot.endTime) - datetime.combine(today, lesson2.timeslot.startTime)
    return timedelta(minutes=0) <= between <= timedelta(minutes=30)

@constraint_provider
def defineConstraints(constraintFactory):
    return [
        # Hard constraints
        roomConflict(constraintFactory),
        teacherConflict(constraintFactory),
        studentGroupConflict(constraintFactory),
        # Soft constraints
        teacherRoomStability(constraintFactory),
        teacherTimeEfficiency(constraintFactory),
        studentGroupSubjectVariety(constraintFactory)
    ]

def roomConflict(constraintFactory):
    # A room can accommodate at most one lesson at the same time.
    return constraintFactory \
            .fromUniquePair(LessonClass,
                [
                    # ... in the same timeslot ..
                    Joiners.equal(lambda lesson: lesson.timeslot),
                    # ... in the same room ...
                    Joiners.equal(lambda lesson: lesson.room)
                ]) \
            .penalize("Room conflict", HardSoftScore.ONE_HARD)


def teacherConflict(constraintFactory):
    # A teacher can teach at most one lesson at the same time.
    return constraintFactory \
                .fromUniquePair(LessonClass,
                        [
                            Joiners.equal(lambda lesson: lesson.timeslot),
                            Joiners.equal(lambda lesson: lesson.teacher)
                        ]) \
                .penalize("Teacher conflict", HardSoftScore.ONE_HARD)

def studentGroupConflict(constraintFactory):
    # A student can attend at most one lesson at the same time.
    return constraintFactory \
            .fromUniquePair(LessonClass,
                [
                    Joiners.equal(lambda lesson: lesson.timeslot),
                    Joiners.equal(lambda lesson: lesson.studentGroup)
                ]) \
            .penalize("Student group conflict", HardSoftScore.ONE_HARD)

def teacherRoomStability(constraintFactory):
    # A teacher prefers to teach in a single room.
    return constraintFactory \
                .fromUniquePair(LessonClass,
                        [Joiners.equal(lambda lesson: lesson.teacher)]) \
                .filter(lambda lesson1, lesson2: lesson1.room != lesson2.room) \
                .penalize("Teacher room stability", HardSoftScore.ONE_SOFT)

def teacherTimeEfficiency(constraintFactory):
    # A teacher prefers to teach sequential lessons and dislikes gaps between lessons.
    return constraintFactory.from_(LessonClass)\
                .join(LessonClass, [Joiners.equal(lambda lesson: lesson.teacher),
                        Joiners.equal(lambda lesson: lesson.timeslot.dayOfWeek)]) \
                .filter(within30Mins) \
                .reward("Teacher time efficiency", HardSoftScore.ONE_SOFT)

def studentGroupSubjectVariety(constraintFactory):
    # A student group dislikes sequential lessons on the same subject.
    return constraintFactory.from_(LessonClass) \
        .join(LessonClass,
                        [
                            Joiners.equal(lambda lesson: lesson.subject),
                            Joiners.equal(lambda lesson: lesson.studentGroup),
                            Joiners.equal(lambda lesson: lesson.timeslot.dayOfWeek)
                        ]) \
        .filter(within30Mins) \
        .penalize("Student group subject variety", HardSoftScore.ONE_SOFT)

The `@constraint_provider` decorator creates a Java `ConstraintProvider` class, allowing OptaPlanner to use it. You can call any python method when evaluating your constraints. 

#### Planning Solution

Finally, there is the planning solution. The planning solution stores references to all the problem facts and planning entities that define the problem. Additionally, it also contain the score of the solution. The planning solution class represent both the problem and the solution; as such, a problem can be viewed as an unintialized planning solution. Here how we define it in Python:

In [5]:
from functools import reduce

# Used to create a better string
def listString(aList):
    def itemConcat(soFar, newItem):
        return soFar + ",\n" + str(newItem)
    if len(aList) == 0:
        return "[]"
    elif len(aList) == 1:
        return "[" + str(aList[0]) + "]"
    else:
        return "[" + reduce(itemConcat, aList[1:], str(aList[0])) + "]"


from optapy import planning_solution, planning_entity_collection_property, problem_fact_collection_property, \
                   value_range_provider, planning_score

@planning_solution
class TimeTable:
    def __init__(self, timeslotList=[], roomList=[], lessonList=[], score=None):
        self.timeslotList = timeslotList
        self.roomList = roomList
        self.lessonList = lessonList
        self.score = score

    @problem_fact_collection_property(Timeslot)
    @value_range_provider("timeslotRange")
    def getTimeslotList(self):
        return self.timeslotList

    @problem_fact_collection_property(Room)
    @value_range_provider("roomRange")
    def value_range_provider(self):
        return self.roomList

    @planning_entity_collection_property(Lesson)
    def getLessonList(self):
        return self.lessonList

    @planning_score(HardSoftScore)
    def getScore(self):
        return self.score

    def setScore(self, score):
        self.score = score

    def __str__(self):
        return "TimeTable(timeSlotList=" + listString(self.timeslotList) + \
               ",\nroomList=" + listString(self.roomList) + ",\nlessonList=" + listString(self.lessonList) + \
               ",\nscore=" + str(self.score.toString()) + ")"

The `@planning_solution` decorator creates a Java class for TimeTable, allowing it to be passed to OptaPlanner.
The `@problem_fact_collection_property` decorator tells OptaPlanner that method returns problem facts (it takes in one required argument: the Python class of the problem fact). Similarly, the `@planning_entity_collection_property` decorator tells OptaPlanner that method returns planning entities (it takes in one required argument: the Python class of the planning entity). The `@value_range_provider` decorator tells OptaPlanner the method provide values for variables. It `range_id` parameter is used determine what planning variable(s) accept values from it. For example, `timeslot` take values from the `timeslotRange`, so it accept values from `getTimeslotList`. Finally, the `@planning_score` decorator tells OptaPlanner the method returns the planning score (how good the solution is). Like with `@planning_variable`, It must be named `get%Score()` and has a corresponding setter `set%Score` (where `%Score` is the name of the score). Its parameter tells OptaPlanner what kind of score it takes.

### Solving

Now that we defined our model and constraints, let create an instance of the problem and solve it.

In [ ]:
from datetime import time

def generateProblem():
    timeslotList = [
        Timeslot(1, "MONDAY", time(hour=8, minute=30), time(hour=9, minute=30)),
        Timeslot(2, "MONDAY", time(hour=9, minute=30), time(hour=10, minute=30)),
        Timeslot(3, "MONDAY", time(hour=10, minute=30), time(hour=11, minute=30)),
        Timeslot(4, "MONDAY", time(hour=13, minute=30), time(hour=14, minute=30)),
        Timeslot(5, "MONDAY", time(hour=14, minute=30), time(hour=15, minute=30)),
        Timeslot(6, "TUESDAY", time(hour=8, minute=30), time(hour=9, minute=30)),
        Timeslot(7, "TUESDAY", time(hour=9, minute=30), time(hour=10, minute=30)),
        Timeslot(8, "TUESDAY", time(hour=10, minute=30), time(hour=11, minute=30)),
        Timeslot(9, "TUESDAY", time(hour=13, minute=30), time(hour=14, minute=30)),
        Timeslot(10, "TUESDAY", time(hour=14, minute=30), time(hour=15, minute=30)),
    ]
    roomList = [
        Room(1, "Room A"),
        Room(2, "Room B"),
        Room(3, "Room C")
    ]
    lessonList = [
        Lesson(1, "Math", "A. Turing", "9th grade"),
        Lesson(2, "Math", "A. Turing", "9th grade"),
        Lesson(3, "Physics", "M. Curie", "9th grade"),
        Lesson(4, "Chemistry", "M. Curie", "9th grade"),
        Lesson(5, "Biology", "C. Darwin", "9th grade"),
        Lesson(6, "History", "I. Jones", "9th grade"),
        Lesson(7, "English", "I. Jones", "9th grade"),
        Lesson(8, "English", "I. Jones", "9th grade"),
        Lesson(9, "Spanish", "P. Cruz", "9th grade"),
        Lesson(10, "Spanish", "P. Cruz", "9th grade"),
        Lesson(11, "Math", "A. Turing", "10th grade"),
        Lesson(12, "Math", "A. Turing", "10th grade"),
        Lesson(13, "Math", "A. Turing", "10th grade"),
        Lesson(14, "Physics", "M. Curie", "10th grade"),
        Lesson(15, "Chemistry", "M. Curie", "10th grade"),
        Lesson(16, "French", "M. Curie", "10th grade"),
        Lesson(17, "Geography", "C. Darwin", "10th grade"),
        Lesson(18, "History", "I. Jones", "10th grade"),
        Lesson(19, "English", "P. Cruz", "10th grade"),
        Lesson(20, "Spanish", "P. Cruz", "10th grade"),
    ]
    lesson = lessonList[0]
    lesson.setTimeslot(timeslotList[0])
    lesson.setRoom(roomList[0])

    return TimeTable(timeslotList, roomList, lessonList)

from optapy import solve
from optapy.types import SolverConfig, Duration
solverConfig = SolverConfig().withEntityClasses(get_class(Lesson)) \
    .withSolutionClass(get_class(TimeTable)) \
    .withConstraintProviderClass(get_class(defineConstraints)) \
    .withTerminationSpentLimit(Duration.ofSeconds(30))

solution = solve(solverConfig, generateProblem())

print(solution)

Which will print a solution that look like the following:

```
TimeTable(timeSlotList=[Timeslot(id=1, dayOfWeek=MONDAY, startTime=08:30:00, endTime=09:30:00),
Timeslot(id=2, dayOfWeek=MONDAY, startTime=09:30:00, endTime=10:30:00),
Timeslot(id=3, dayOfWeek=MONDAY, startTime=10:30:00, endTime=11:30:00),
Timeslot(id=4, dayOfWeek=MONDAY, startTime=13:30:00, endTime=14:30:00),
Timeslot(id=5, dayOfWeek=MONDAY, startTime=14:30:00, endTime=15:30:00),
Timeslot(id=6, dayOfWeek=TUESDAY, startTime=08:30:00, endTime=09:30:00),
Timeslot(id=7, dayOfWeek=TUESDAY, startTime=09:30:00, endTime=10:30:00),
Timeslot(id=8, dayOfWeek=TUESDAY, startTime=10:30:00, endTime=11:30:00),
Timeslot(id=9, dayOfWeek=TUESDAY, startTime=13:30:00, endTime=14:30:00),
Timeslot(id=10, dayOfWeek=TUESDAY, startTime=14:30:00, endTime=15:30:00)],
roomList=[Room(id=1, name=Room A),
Room(id=2, name=Room B),
Room(id=3, name=Room C)],
lessonList=[Lesson(id=1, timeslot=Timeslot(id=1, dayOfWeek=MONDAY, startTime=08:30:00, endTime=09:30:00), room=Room(id=1, name=Room A), teacher=A. Turing, subject=Math, studentGroup=9th grade),
Lesson(id=2, timeslot=Timeslot(id=2, dayOfWeek=MONDAY, startTime=09:30:00, endTime=10:30:00), room=Room(id=1, name=Room A), teacher=A. Turing, subject=Math, studentGroup=9th grade),
Lesson(id=3, timeslot=Timeslot(id=3, dayOfWeek=MONDAY, startTime=10:30:00, endTime=11:30:00), room=Room(id=1, name=Room A), teacher=M. Curie, subject=Physics, studentGroup=9th grade),
Lesson(id=4, timeslot=Timeslot(id=10, dayOfWeek=TUESDAY, startTime=14:30:00, endTime=15:30:00), room=Room(id=1, name=Room A), teacher=M. Curie, subject=Chemistry, studentGroup=9th grade),
Lesson(id=5, timeslot=Timeslot(id=5, dayOfWeek=MONDAY, startTime=14:30:00, endTime=15:30:00), room=Room(id=1, name=Room A), teacher=C. Darwin, subject=Biology, studentGroup=9th grade),
Lesson(id=6, timeslot=Timeslot(id=6, dayOfWeek=TUESDAY, startTime=08:30:00, endTime=09:30:00), room=Room(id=1, name=Room A), teacher=I. Jones, subject=History, studentGroup=9th grade),
Lesson(id=7, timeslot=Timeslot(id=7, dayOfWeek=TUESDAY, startTime=09:30:00, endTime=10:30:00), room=Room(id=1, name=Room A), teacher=I. Jones, subject=English, studentGroup=9th grade),
Lesson(id=8, timeslot=Timeslot(id=8, dayOfWeek=TUESDAY, startTime=10:30:00, endTime=11:30:00), room=Room(id=1, name=Room A), teacher=I. Jones, subject=English, studentGroup=9th grade),
Lesson(id=9, timeslot=Timeslot(id=9, dayOfWeek=TUESDAY, startTime=13:30:00, endTime=14:30:00), room=Room(id=1, name=Room A), teacher=P. Cruz, subject=Spanish, studentGroup=9th grade),
Lesson(id=10, timeslot=Timeslot(id=4, dayOfWeek=MONDAY, startTime=13:30:00, endTime=14:30:00), room=Room(id=1, name=Room A), teacher=P. Cruz, subject=Spanish, studentGroup=9th grade),
Lesson(id=11, timeslot=Timeslot(id=3, dayOfWeek=MONDAY, startTime=10:30:00, endTime=11:30:00), room=Room(id=2, name=Room B), teacher=A. Turing, subject=Math, studentGroup=10th grade),
Lesson(id=12, timeslot=Timeslot(id=4, dayOfWeek=MONDAY, startTime=13:30:00, endTime=14:30:00), room=Room(id=2, name=Room B), teacher=A. Turing, subject=Math, studentGroup=10th grade),
Lesson(id=13, timeslot=Timeslot(id=5, dayOfWeek=MONDAY, startTime=14:30:00, endTime=15:30:00), room=Room(id=2, name=Room B), teacher=A. Turing, subject=Math, studentGroup=10th grade),
Lesson(id=14, timeslot=Timeslot(id=6, dayOfWeek=TUESDAY, startTime=08:30:00, endTime=09:30:00), room=Room(id=2, name=Room B), teacher=M. Curie, subject=Physics, studentGroup=10th grade),
Lesson(id=15, timeslot=Timeslot(id=1, dayOfWeek=MONDAY, startTime=08:30:00, endTime=09:30:00), room=Room(id=2, name=Room B), teacher=M. Curie, subject=Chemistry, studentGroup=10th grade),
Lesson(id=16, timeslot=Timeslot(id=2, dayOfWeek=MONDAY, startTime=09:30:00, endTime=10:30:00), room=Room(id=2, name=Room B), teacher=M. Curie, subject=French, studentGroup=10th grade),
Lesson(id=17, timeslot=Timeslot(id=7, dayOfWeek=TUESDAY, startTime=09:30:00, endTime=10:30:00), room=Room(id=2, name=Room B), teacher=C. Darwin, subject=Geography, studentGroup=10th grade),
Lesson(id=18, timeslot=Timeslot(id=9, dayOfWeek=TUESDAY, startTime=13:30:00, endTime=14:30:00), room=Room(id=3, name=Room C), teacher=I. Jones, subject=History, studentGroup=10th grade),
Lesson(id=19, timeslot=Timeslot(id=8, dayOfWeek=TUESDAY, startTime=10:30:00, endTime=11:30:00), room=Room(id=2, name=Room B), teacher=P. Cruz, subject=English, studentGroup=10th grade),
Lesson(id=20, timeslot=Timeslot(id=10, dayOfWeek=TUESDAY, startTime=14:30:00, endTime=15:30:00), room=Room(id=2, name=Room B), teacher=P. Cruz, subject=Spanish, studentGroup=10th grade)],
score=0hard/2soft)
```